# Second session: multilayer perceptron MLP


In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

ImportError: No module named 'tensorflow'

Reminder :The MNIST dataset is a set of greyscale images with 28*28=784 pixels.
Each image is a number between 0 and 9.
The goal of the practical is to obtain a classifier that can correctly 
classify the images into the 10 classes.
The MNIST data consists of 55000 train images and labels, 5000 observations
validation set and 10000 observations in the test set.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, [None, 784])#input
y_ = tf.placeholder(tf.float32, [None, 10])#labels

## TP2 2: Deep Learning
for more information about convolutional network : http://cs231n.github.io/convolutional-networks/#conv

or a youtube video ;) : https://www.youtube.com/watch?v=bEUX_56Lojc

The purpose: Getting 91% accuracy on MNIST is bad. It's almost embarrassingly bad. In 
this practical, we'll fix that, jumping from a very simple model to something
moderately sophisticated: a small convolutional neural network. This will get 
us to around 99.2% accuracy -- not state of the art, but respectable.

### Weight Initialization

To create this model, we're going to need to create a lot of weights and 
biases. One should generally initialize weights with a small amount of 
noise for symmetry breaking, and to prevent 0 gradients. Since we're using 
ReLU neurons, it is also good practice to initialize them with a slightly 
positive initial bias to avoid "dead neurons." Instead of doing this 
repeatedly while we build the model, let's create two handy functions to 
do it for us.

#### Q1 : Why is symmetry breaking important ? What is a ReLU neuron and why is "dead neurons" a specific problem ?

In [3]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling
 operations. How do we handle the boundaries? What is our stride size? Here
we will use the simplest possible options. Our convolutions uses a stride 
of one and are zero padded so that the output is the same size as the input.
Our pooling is max pooling over 2x2 blocks. To keep our code cleaner, let's
also abstract those operations into functions.

In [4]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

#### First Convolutional Layer

We can now implement our first layer. It will consist of convolution, 
followed by max pooling. The convolutional will compute 32 features for 
each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The 
first two dimensions are the patch size, the next is the number of input 
channels, and the last is the number of output channels. We will also have
 a bias vector with a component for each output channel.

In [5]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape x to a 4d tensor, with the second 
and third dimensions corresponding to image width and height, and the final
 dimension corresponding to the number of color channels.

In [6]:
x_image = tf.reshape(x, [-1,28,28,1])

#### Q2.1: What does the -1 in the shape vector mean? Use the documentation. Why do we need to reshape x compared to our first algorithm ?

We then convolve x_image with the weight tensor, add the bias, apply the
ReLU function, and finally max pool.

In [7]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

The zero-padding and the stride of 1 we selected ensure that the output
remains of the same size as the input image. 

#### Q2.2: What size is the patch ? How many lines/columns of zero-padding are there ? What would be the size of the output image if there was no padding ?

#### Q2.3: What is the dimension of this layer ? What would it be if we did not do weight sharing ?

#### Second Convolutional Layer

In order to build a deep network, we stack several layers of this type. The
second layer will have 64 features for each 5x5 patch.
#### Q2.4: Code the second convolutional layer.

In [8]:
#answer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.
 
#### Q2.5: Code the densely connected layer.

In [9]:
#answer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

Finally, we add a softmax layer, just like for the one layer softmax 
regression above.

#### Q2.6: code the softmax layer.

In [10]:

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)


cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(2200):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1]})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

step 0, training accuracy 0.16
step 100, training accuracy 0.82
step 200, training accuracy 0.9
step 300, training accuracy 0.94
step 400, training accuracy 1
step 500, training accuracy 0.92
step 600, training accuracy 0.98
step 700, training accuracy 0.94
step 800, training accuracy 0.94
step 900, training accuracy 1
step 1000, training accuracy 0.98
step 1100, training accuracy 1
step 1200, training accuracy 0.98
step 1300, training accuracy 1
step 1400, training accuracy 1
step 1500, training accuracy 0.96
step 1600, training accuracy 0.98
step 1700, training accuracy 1
step 1800, training accuracy 0.98
step 1900, training accuracy 0.94
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98


Q2.7: How much observations are used at each step ? How much are used
in the loop ?
The classifier can be trained longer and it will still improve the performance:
with 20000 iterations the accuracy is 0.992

In [2]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels}))

NameError: name 'accuracy' is not defined

#### Q2.7: Take a look at some misclassified digits. Look at the softmax output. What do you think of the performance of the algorithm? Are you better ?

In [6]:
correct, softmax=sess.run([correct_prediction,y_conv],feed_dict={
    x: mnist.test.images[range(1000),], y_: mnist.test.labels[range(1000),]})

np.where(correct==False)
plt.matshow(np.reshape(mnist.test.images[247,],[28,28]))
mnist.test.labels[247,]
softmax[247,]



#### Q2.8: Vizualize all of the 32 filters of the first hidden layer in a single figure.

In [5]:
for i in range(1,33):
    plt.subplot(8,4,i)
    plt.imshow(0.2+sess.run(W_conv1)[:,:,0,i-1], cmap=plt.cm.BuPu_r)
plt.colorbar()

#### Q2.9: What is overfitting ? What ways do we have to limit overfitting for neural networks ? What is an additional advantage of drop-out ?

In [16]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#### Q2.10: recode the training step with a drop-out of 0.5

In [7]:
#answer
for i in range(3000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

close session

In [20]:
sess.close()

To go further :
Mnist is a baby problem compared to real life image recognition problem. What is ImageNet ? What are the best performances obtained on it and by what architecture ?
You can also read this : http://karpathy.github.io/2014/09/02/what-i-learned-from-competing-against-a-convnet-on-imagenet/